In [2]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 40.9 MB/s eta 0:00:00


In [3]:
import math
import struct
import time
import random
import string
from z3 import *

# MD5 Implementation

In [4]:
class Hash_value:
    def __init__(self, hashvalue_hexa):
        self.hashvalue_hexa = hashvalue_hexa
        self.hashvalue_bits = bin(int(hashvalue_hexa, 16))[2:].zfill(len(hashvalue_hexa) * 4)
        self.hashvalue_integer =  int(hashvalue_hexa, 16)

class MD5_Hash:
    """
    This class implements the MD5 Algorithm and its light-weight variant as a z3 implementation
    set:
    type: light (for MD5 light) or regular (for MD5)
    """
    # Constructor (__init__)
    def __init__(self, type = "regular",):
        # Set all Variables and Constants dependent on wordsize_bit
        if (type == "regular"):
            self.wordsize_bit = 32
            self.length_bits = 'Q' # 64 bit / 8 Byte
        elif (type == "medium"):
            self.wordsize_bit = 16
            self.length_bits = 'I' # 32 bit / 4 Byte
        elif (type == "light"):
            self.wordsize_bit = 8
            self.length_bits = 'H' # 16 bit / 2 Byte
        else: raise ValueError("hash type must be either light or regular")
        self.wordsize_byte = int(self.wordsize_bit / 8)
        self.blocksize_byte = self.wordsize_byte * 16
        self.hexa = (1 << (self.wordsize_byte * 8)) - 1
        a0, b0, c0, d0 = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476
        self.Ks = [int(abs(math.sin(i + 1)) * (2**self.wordsize_bit)) & self.hexa for i in range(64)]
        self.Ss = list(map(lambda s: s % self.wordsize_bit,[7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4))
        self.a0,self.b0,self.c0,self.d0 = map(lambda x: int(hex(x)[2:].upper()[:(self.wordsize_byte * 2)],16), [a0, b0, c0, d0])
        self.hashlength_bit = 4* self.wordsize_bit
        self.hashlength_byte = 4* self.wordsize_byte
    #------------------------------------------------------------------------------------------------------------------------
    #Pre-Processing
    #convert Type
    def convertType(self,m):
        # Make sure m is converted to byte value
        if isinstance(m, str) and all(c in '01' for c in m):  # If input is a bit string
            bv = [m[i:i+8] for i in range(0, len(m), 8)]
            message = ''.join(chr(int(b, 2)) for b in bv)
        elif isinstance(m, str):  # If input is a message (string)
            message = m
        else:
            raise ValueError("Input must be either a message (string) or a valid bit string.")
        return message.encode('utf-8') # bytes

    #Padding Function
    def pad(self,m):
        bytes = self.convertType(m)
        original_length_bits = len(bytes) * 8
        bytes += b'\x80' #appends 10000000
        padding_value = self.blocksize_byte - 8 # in byte
        length_bits = 'Q' # 8 Byte
        if (self.wordsize_bit == 8):
            padding_value = self.blocksize_byte - 2 # in byte
            length_bits = 'H' # 2 Byte
        while len(bytes)%(self.blocksize_byte)!=padding_value:
            bytes += b'\x00' #appends 00000000
        bytes += struct.pack(f'<{length_bits}', original_length_bits) # appends 16 bit /64 bit representation in litte-endian-format of length
        return bytes

    def split_bytes(self, data, chunk_size):
      return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

    #------------------------------------------------------------------------------------------------------------------------
    # Auxiliary Functions
    # Rotation function (rotate left)
    def rotate_left(self, x, n):
        # Perform a left rotation on a 32-bit integer 'x' by 'n' positions
        x &= self.hexa
        return ((x << n) | (x >> (self.wordsize_bit - n))) & self.hexa

    #------------------------------------------------------------------------------------------------------------------------
    #Hashing
    def processblockZ3(self, solver, M, inits = None , p = '', rounds = 64):
        if rounds > 64 or not isinstance(rounds, int):
            raise ValueError("rounds must be int < 64")
        if inits is None:
            inits = [self.a0, self.b0, self.c0, self.d0]
        # Declaring constants and values:
        bitsize = 2 * self.wordsize_bit

        S = [BitVec(f'S{p}_{i}', bitsize) for i in range(rounds)]
        K = [BitVec(f'K{p}_{i}', bitsize) for i in range(rounds)]

        A = [BitVec(f'A{p}_{i}', bitsize) for i in range(rounds+1)]
        B = [BitVec(f'B{p}_{i}', bitsize) for i in range(rounds+1)]
        C = [BitVec(f'C{p}_{i}', bitsize) for i in range(rounds+1)]
        D = [BitVec(f'D{p}_{i}', bitsize) for i in range(rounds+1)]


        #Initializing constants and values:
        for i in range(rounds):
            solver.add(K[i] == self.Ks[i])
            solver.add(S[i] == self.Ss[i])

        solver.add(A[0] == inits[0])
        solver.add(B[0] == inits[1])
        solver.add(C[0] == inits[2])
        solver.add(D[0] == inits[3])


        #Hashing:
        f = [BitVec(f'f{p}_{i}', bitsize) for i in range(rounds)]
        for i in range(rounds):
            if( i < 16):
                solver.add(f[i] == (B[i] & C[i]) | (~B[i] & D[i]))
                g = i
            elif( i < 32):
                solver.add(f[i] == (B[i] & D[i]) | (C[i] & ~D[i]))
                g = (5 * i + 1) % 16
            elif( i < 48):
                solver.add(f[i] == B[i] ^ C[i] ^ D[i])
                g = (3 * i + 5) % 16
            elif( i < 64):
                solver.add(f[i] == C[i] ^ (B[i] | ~D[i]))
                g = (7 * i) % 16
            # Ensure extracted bits are within valid range
            start = self.wordsize_byte * g
            end = self.wordsize_byte * g + self.wordsize_byte
            if (end - start == 1): word = M[start]
            else: word = Concat(*[M[start + i] for i in range(end-start)])

            solver.add(D[i+1] == C[i])
            solver.add(C[i+1] == B[i])
            solver.add(B[i+1] == (B[i] + self.rotate_left((A[i] + f[i] + K[i] + ZeroExt(bitsize - word.size(),word)),S[i])) & self.hexa)
            solver.add(A[i+1] == D[i])
        return A[rounds],B[rounds],C[rounds],D[rounds]

    def hash(self,m):
        blocks = self.pad(m)
        # Process each block
        A_final,B_final,C_final,D_final = self.a0, self.b0, self.c0, self.d0
        for block in self.split_bytes(blocks, self.blocksize_byte):
            solver = Solver()
            M = [BitVec(f'M1_{i}', self.wordsize_bit // 4) for i in range(self.blocksize_byte)]
            for i in range(self.blocksize_byte):
              solver.add(M[i] == block[i])
            #Compute Hash
            A,B,C,D = self.processblockZ3(solver, M, [A_final,B_final,C_final,D_final])
            if solver.check() == sat:
                model = solver.model()
                A_final = (A_final + model[A].as_long()) & self.hexa
                B_final = (B_final + model[B].as_long()) & self.hexa
                C_final = (C_final + model[C].as_long()) & self.hexa
                D_final = (D_final + model[D].as_long()) & self.hexa
            else:
                print("No solution - hashing")
                return 0,0,0,0
        return A_final, B_final, C_final, D_final

    #digest
    def digest(self, m):
        A,B,C,D = self.hash(m)
        raw = sum(value << (self.wordsize_bit * i) for i, value in enumerate([A,B,C,D])).to_bytes(16, byteorder='little')
        hashvalue_hexa = '{:0{width}x}'.format(int.from_bytes(raw[:self.hashlength_byte], byteorder='big'), width = self.wordsize_bit)
        return Hash_value(hashvalue_hexa)

h1 = MD5_Hash().digest("Hello")
print(f"Hash as Bitstring: {h1.hashvalue_bits} - Hash as Hexastring: {h1.hashvalue_hexa} - Hash as integervalue: {h1.hashvalue_integer}")

Hash as Bitstring: 00100100001111101110100010010001111101011000100011001100010111110011101110011011100010000001111011101111000010111000100000000111 - Hash as Hexastring: 243ee891f588cc5f3b9b881eef0b8807 - Hash as integervalue: 48178847336586948930568751920106407943


# Collisionfinder

In [ ]:
# to find a collision for a hash in md5 light only one block is needed
def find_collision(hash_type = "regular", collisionpercentage = 100,rounds = 64, timeout = 7200):
    """
    This class implements the collision attack for the MD5 Algorithm and its light-weight variant as a z3 (SAT) implementation
    set:
      type: light (for MD5 light) or regular (for MD5)
      collision_percentage: 100%, 75%, 50%, 25% (default: 100%)
      timeout: in seconds (default: 2h)
      rounds: 8,16,32,64 (default: 64)
    """
    if rounds > 64 or type(rounds) is not int:
            raise ValueError("rounds must be int < 64")
    hash_function = MD5_Hash(hash_type)
    collisionsolver = Solver()
    collisionsolver.set("timeout", timeout * 1000)
    M1 = [BitVec(f'M1_{i}', hash_function.wordsize_bit // 4) for i in range(hash_function.blocksize_byte)]
    M2 = [BitVec(f'M2_{i}', hash_function.wordsize_bit // 4) for i in range(hash_function.blocksize_byte)]

    A1,B1,C1,D1 = hash_function.processblockZ3(collisionsolver, M1, None, '1',rounds)
    A2,B2,C2,D2 = hash_function.processblockZ3(collisionsolver, M2, None, '2',rounds)

    if collisionpercentage == 100:
       collisionsolver.add(And(A1 == A2, B1 == B2, C1==C2, D1 ==D2))
    elif collisionpercentage == 75:
        collisionsolver.add(And(A1 == A2, B1 == B2, C1==C2))
    elif collisionpercentage == 50:
        collisionsolver.add(And(A1 == A2, B1 == B2))
    elif collisionpercentage == 25:
        collisionsolver.add((A1 == A2))

    collisionsolver.add(Or([M1[i] != M2[i] for i in range(16)]))
    print(f"Number Constraints: {len(collisionsolver.assertions())}")
    if collisionsolver.check() == sat:
        return collisionsolver
    print("timeout")
    return collisionsolver
